In [1]:
import json
from datetime import datetime
from tqdm import tqdm
from shroom_classifier_v1_persona import ShroomClassifier

In [2]:
def run_experiment(cp, reference, submission):
    data = json.load(open(reference, "r"))
    predictions = []
    for dp in tqdm(data):
        prediction = cp.classify(dp)
        predictions.append(prediction)
    json.dump(predictions, open(submission, "w+"))

In [3]:
RUN = "v1-persona"
MODEL = "gpt-3.5-turbo"
TEMP = 0.7
DATE = datetime.utcnow().date().isoformat()

In [4]:
CLASSIFIER = ShroomClassifier(model_name=MODEL, temperature=TEMP)
run_experiment(CLASSIFIER, 'reference/val.model-agnostic.json', 'submission/val.model-agnostic.json')
run_experiment(CLASSIFIER, 'reference/val.model-aware.json', 'submission/val.model-aware.json')

/Users/bradleyallen/Documents/GitHub/shroom/env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
100%|██████████| 501/501 [40:56<00:00,  4.90s/it]   


In [5]:
! python score.py --is_val submission reference scores/score-{MODEL}-{RUN}-{DATE}.txt

In [6]:
record = { "date": DATE, "version": RUN, "model": MODEL, "temperature": TEMP }
score_filename = f'scores/score-{MODEL}-{RUN}-{DATE}.txt'
score_file = open(score_filename, 'r')
for score in score_file.readlines():
    pair = score.strip().split(':')
    record[pair[0]] = float(pair[1])
record["avg_acc"] = (record["agnostic_acc"] + record["aware_acc"]) / 2.
record["avg_rho"] = (record["agnostic_rho"] + record["aware_rho"]) / 2.
record["val.model-aware"] = json.load(open('submission/val.model-aware.json', 'r'))
record["val.model-agnostic"] = json.load(open('submission/val.model-agnostic.json', 'r'))
records = json.load(open('validation_runs.json', 'r'))
records.append(record)
json.dump(records, open('validation_runs.json', 'w+'))

In [7]:
! mv submission/val.model-agnostic.json results/val.model-agnostic.{MODEL}-{RUN}-{DATE}
! mv submission/val.model-aware.json results/val.model-aware.{MODEL}-{RUN}-{DATE}